# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
\mu = \int r ρ(r) dr
$$
with respect to a small uniform electric field ``E = -x``.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of ``a`` bohrs
He = ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))
atoms = [He => [[1/2; 1/2; 1/2]]]  # Helium at the center of the box

kgrid = [1, 1, 1]  # no kpoint sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    d = sum(rr .* ρ) * basis.dvol
end;

## Polarizability by finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms; symmetries=false)
basis = PlaneWaveBasis(model, Ecut; kgrid=kgrid)
res = self_consistent_field(basis, tol=tol)
μref = dipole(basis, res.ρ)

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   Diag
---   ---------------   ---------   --------   ----
  1   -2.769875367758         NaN   2.98e-01    8.0 
  2   -2.771179518450   -1.30e-03   4.94e-02    1.0 
  3   -2.771212464901   -3.29e-05   2.36e-03    1.0 
  4   -2.771212978613   -5.14e-07   1.46e-04    2.0 
  5   -2.771212981467   -2.85e-09   4.48e-05    2.0 


-0.00013721148168794697

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms; extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε, Ecut; kgrid=kgrid)
res_ε = self_consistent_field(basis_ε, tol=tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   Diag
---   ---------------   ---------   --------   ----
  1   -2.770004446278         NaN   2.98e-01    8.0 
  2   -2.771272521628   -1.27e-03   4.84e-02    1.0 
  3   -2.771299818005   -2.73e-05   2.93e-03    1.0 
  4   -2.771300329326   -5.11e-07   4.40e-04    2.0 
  5   -2.771300360195   -3.09e-08   1.38e-04    2.0 
  6   -2.771300362350   -2.15e-09   7.44e-06    2.0 


0.017617224547225606

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013721148168794697
Displaced dipole:  0.017617224547225606
Polarizability :   1.7754436028913554


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Polarizability by linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, ``\chi_0`` is the
independent-particle polarizability, and ``K`` the
Hartree-exchange-correlation kernel. We denote with ``dV_{\rm ext}`` an external
perturbing potential (like in this case the uniform electric field). Then:
$$
d\rho = \chi_0 dV = \chi_0 (dV_{\rm ext} + K d\rho),
$$
which implies
$$
d\rho = (1-\chi_0 K)^-1 \chi_0 dV_{\rm ext}.
$$
From this we identify the polarizability operator to be ``\chi = (1-\chi_0 K)^-1 \chi_0``.
Numerically, we apply ``\chi`` to ``dV = -x`` by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply (1- χ0 K)
function dielectric_operator(dρ)
    dv = apply_kernel(basis, dρ; ρ=res.ρ)
    χ0dv = apply_χ0(res.ham, res.ψ, res.εF, res.eigenvalues, dv)
    dρ - χ0dv
end

# dVext is the potential from a uniform field interacting with the dielectric dipole
# of the density.
dVext = [-a * (r[1] - 1/2) for r in r_vectors(basis)]
dVext = cat(dVext; dims=4)

# Apply χ0 once to get non-interacting dipole
dρ_nointeract = apply_χ0(res.ham, res.ψ, res.εF, res.eigenvalues, dVext)

# Solve Dyson equation to get interacting dipole
dρ = linsolve(dielectric_operator, dρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, dρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, dρ))")

┌ Info: GMRES linsolve in iter 1; step 1: normres = 2.490534861322e-01
└ @ KrylovKit /home/runner/.julia/packages/KrylovKit/YPiz7/src/linsolve/gmres.jl:55
┌ Info: GMRES linsolve in iter 1; step 2: normres = 3.703079760748e-03
└ @ KrylovKit /home/runner/.julia/packages/KrylovKit/YPiz7/src/linsolve/gmres.jl:89
┌ Info: GMRES linsolve in iter 1; step 3: normres = 8.576710398749e-05
└ @ KrylovKit /home/runner/.julia/packages/KrylovKit/YPiz7/src/linsolve/gmres.jl:89
┌ Info: GMRES linsolve in iter 1; step 4: normres = 7.028707278266e-06
└ @ KrylovKit /home/runner/.julia/packages/KrylovKit/YPiz7/src/linsolve/gmres.jl:89
┌ Info: GMRES linsolve in iter 1; step 5: normres = 6.802658025032e-07
└ @ KrylovKit /home/runner/.julia/packages/KrylovKit/YPiz7/src/linsolve/gmres.jl:89
┌ Info: GMRES linsolve in iter 1; step 6: normres = 1.876729531822e-09
└ @ KrylovKit /home/runner/.julia/packages/KrylovKit/YPiz7/src/linsolve/gmres.jl:89
┌ Info: GMRES linsolve in iter 1; step 7: normres = 1.964057362748e-11

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.